# Librerías

In [88]:
import re 
import pickle
import numpy as np
from glob import glob

# Funciones útiles

In [89]:
def limpiar_text(text: str)-> str:
    text = text.strip()
    text = text.replace('(','').replace(')', '')
    text = re.sub(r'\s+', ' ', text)

    text = re.sub(r'\s+[bcd]+\s+', ' ', text)
    text = re.sub(r' a ([A-Z])', r' \1', text)
    return text

In [90]:
def partir_versiculos(text: str) -> list:
    versiculos = np.array([int(i) for i in re.findall(r'\n(\d+)', text)])
    versiculos = range(1, np.max(versiculos)+1)

    versiculos_idioma = []

    for i in versiculos[:-1]:
        if i ==  versiculos[-1]:
            ver = '\n '.join(text.split('\n' + str(i))[1:])
        else:
            ver = '\n '.join(text.split('\n' + str(i))[1:])
            ver = ver.split('\n' + str(i+1))[0]
    
        ver = limpiar_text(ver)
        versiculos_idioma.append(ver)
    
    return versiculos_idioma

In [91]:
def clean_raw_text(text):
    text = re.sub(r'\n (\d+)', r'\n\1', text)
    return text 

# Limpieza de los versículos que están juntos

In [92]:
lista = [
    ('MAT', 28, 'Mateo'),
    ('MRK', 16, 'Marcos'),
    ('LUK', 24, 'Lucas'),
    ('JHN', 21, 'Juan'),
    ('ACT', 28, 'Hechos de los apostoles'),
    ('ROM', 16,'Roma'),
    ('1CO', 16, 'Corintio'),
    ('2CO', 13, 'Corintio2'),
    ('GAL', 6, 'Galacia'),
    ('EPH', 6, 'Efeso'),
    ('PHP', 4, 'Filipos'),
    ('COL', 4, 'Colosa'),
    ('1TH', 5, 'Tesalonica'),
    ('2TH', 3, '2Tesalonica'),
    ('1TI', 6, 'Timoteo'),
    ('2TI', 4, '2Timoteo'),
    ('TIT', 3, 'Tito'),
    ('PHM', 1, 'Filemon'),
    ('HEB', 13, 'Hebreokana'),
    ('JAS', 5, 'Santiago'),
    ('1PE', 5, 'Pedro'),
    ('2PE', 3, '2Pedro'),
    ('1JN', 5, '1 Juan'),
    ('2JN', 1, '2 Juan'),
    ('3JN', 1, '3 Juan'),
    ('JUD', 1, 'Judas'),
    ('REV', 22, 'Apocalipsis')
]

In [93]:
 # Buscamos los versículos que es Wayuu están pegados de la forma 1-5
# Esto quiere decir que deben juntar todos en un solo versículo (1)
# Y en el correspondiente de español se deben 

sobre_escribir = {}

for cod, num, folder in lista:
    traduccion = []
    for i in range(1,num+1):
        path = f'../data/biblia/{folder}/'

        with open(path + f'{cod} {i} - esp.txt', 'r') as f:
            esp = ''.join(f.readlines())
        with open(path + f'{cod} {i} - wayuu.txt', 'r') as f:
            wayuu = ''.join(f.readlines())

        patron = r'\n(\d+)-\n(\d+)'
        coincidencias_wayuu = re.findall(patron, wayuu)

        if coincidencias_wayuu:
            for coincidencia in coincidencias_wayuu:
                numero1, numero2 = coincidencia
                # Reemplazar la coincidencia por el primer número
                wayuu = wayuu.replace(f"{numero1}-\n{numero2}", str(numero1), 1)

                for j in range(int(numero1) + 1, int(numero2)+1):
                    esp = esp.replace(str(j), "", 1)

            sobre_escribir[path + f'{cod} {i} - esp.txt'] = esp
            sobre_escribir[path + f'{cod} {i} - wayuu.txt'] =  wayuu

In [94]:
for k, v in sobre_escribir.items():
    with open(k, 'w') as f:
        f.write(v)

# Hacer merge entre español y wayuu

Para los que aparecen errores (no coincide el número de versículos se revisa a mano)

In [95]:
borrar = [
    ('../data/biblia/Mateo/MAT 6 - esp.txt', '42 y par.; Is'),
    ('../data/biblia/Mateo/MAT 28 - esp.txt','19 y par.; Lc'),
    ('../data/biblia/Mateo/MAT 28 - esp.txt','48; etc.; Rm'),
    ('../data/biblia/Lucas/LUK 5 - esp.txt','36 y par.; Mc'),
    ('../data/biblia/Lucas/LUK 5 - esp.txt','46.Curación de un paralítico'),
    ('../data/biblia/Lucas/LUK 19 - esp.txt','55 y par.; Jn'),
    ('../data/biblia/Lucas/LUK 19 - esp.txt','55 y par.; Jn'),
    ('../data/biblia/Juan/JHN 13 - esp.txt', '45 y par.; Rm'),
    ('../data/biblia/Juan/JHN 13 - esp.txt', '45 y par.; Rm'),
    ('../data/biblia/Juan/JHN 21 - esp.txt', '24.Jesús y Pedro'),
    ('../data/biblia/Juan/JHN 21 - esp.txt', '75 y par.; Hch'),
    ('../data/biblia/Hechos de los apostoles/ACT 1 - esp.txt', '33.La ascensión de Jesús'),
    ('../data/biblia/Hechos de los apostoles/ACT 1 - esp.txt', '64 y par.; Mc'),
    ('../data/biblia/Hechos de los apostoles/ACT 1 - esp.txt', '7.Elección del sucesor de Judas'),
    ('../data/biblia/Hechos de los apostoles/ACT 13 - esp.txt', '11 y par.; Jn'),
    ('../data/biblia/Hechos de los apostoles/ACT 13 - esp.txt', '60 y par.; Lc'),
    ('../data/biblia/Hechos de los apostoles/ACT 24 - esp.txt', '30.Pablo continúa en prisión'),
    ('../data/biblia/Apocalipsis/REV 1 - esp.txt', '22.Visión de Cristo glorioso'),
    ('../data/biblia/Apocalipsis/REV 22 - esp.txt', '25.Epílogo Estoy a punto de llegar'),
]

In [96]:
for path, borr in borrar:
    with open(path, 'r') as f:
        esp = ''.join(f.readlines())
    esp = esp.replace(borr, '')

    with open(path, 'w') as f:
        f.write(esp)

In [97]:
filas_borrar = [
    ('../data/biblia/Corintio2/2CO 13 - esp.txt', ["13", "14"]),
    ('../data/biblia/Corintio2/2CO 13 - wayuu.txt', ["13", "14"]),
    ('../data/biblia/Hechos de los apostoles/ACT 19 - esp.txt', ["40", "41"]),
    ('../data/biblia/Hechos de los apostoles/ACT 19 - wayuu.txt', ["40", "41"]),
]

In [98]:
for path, list_init in filas_borrar:
    with open(path, 'r') as f:
        esp = f.readlines()
        
    esp = ''.join([l for l in esp if not l.strip().startswith(list_init[0]) and not l.startswith(list_init[1]) ])

    with open(path, 'w') as f:
        f.write(esp)

In [99]:
for cod, num, folder in lista:
    traduccion = []
    for i in range(1,num+1):
        path = f'../data/biblia/{folder}/'

        with open(path + f'{cod} {i} - esp.txt', 'r') as f:
            esp = ''.join(f.readlines())
        with open(path + f'{cod} {i} - wayuu.txt', 'r') as f:
            wayuu = ''.join(f.readlines())

        wayuu = clean_raw_text(wayuu)
        esp = clean_raw_text(esp)

        wayuu = partir_versiculos(wayuu)
        esp = partir_versiculos(esp)

        if len(wayuu) == len(esp):
            for w, e in zip(wayuu, esp): 
                traduccion.append({'wayuu': w, 'esp': e})
        else:
            print(f'Revisar cap {i} de {cod}',  len(wayuu), len(esp))

    with open(path + 'results.pickle', 'wb') as f:
        pickle.dump(traduccion, f)

# Revisión y limpieza de traducciones